<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Tutorial

FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).

### Installation Test
Run the following cell to attempt an import of the package and check if it is properly installed.

In [87]:
import fastoad

## 2. Working with user files

The FAST-OAD environment uses two types of user files:

* Configuration files (**.toml**):

    The configuration example file used for the tutorial can be found [here](./workdir/oad_process.toml).

    FastProblem instances require a configuration file. The usage of this type of file is describe in the [README](https://github.com/fast-aircraft-design/FAST-OAD) of the repository.

    
* System variables files (**.xml**):

    These file regroup the information of the variables involved in the system model. There are two types of system variables files:
    
    1 - INPUT_FILE: The input file contains the global inputs values required to run all the model. The user is free to modify the values of the variables in order that these new values are considered during a model run.
    
    2 - OUTPUT_FILE: The output file contains all the variables (inputs + outputs) values obtained after a model run.
    

![User Files](./img/user_files_arch.svg "User Files Architecture")

In [1]:
import os.path as pth
import openmdao.api as om
from fastoad import api
import logging

DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.toml')
REF_VALUES_FILE = pth.join(DATA_FOLDER_PATH, 'CeRAS01_baseline.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

Unable to import mpi4py. Parallel processing unavailable.


In [2]:
api.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

INFO    : Sample configuration written in workdir\oad_process.toml


In [90]:
api.generate_inputs(CONFIGURATION_FILE, REF_VALUES_FILE, overwrite=True)

INFO    : Problem inputs written in D:\cdavid\PyCharmProjects\FAST-OAD\notebooks\tutorial\workdir\problem_inputs.xml


In [91]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
print(N2_FILE)
IFrame(src=N2_FILE, width='100%', height='500px')

INFO    : N2 diagram written in workdir\n2.html


workdir\n2.html


## 3. Running your first MDA

In [92]:
# problem.set_solver_print(level=2)
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)
print('Input max thrust: %.0f' % eval_problem['propulsion:MTO_thrust'])
print('Cruise thrust rate: %.2f' % eval_problem['propulsion:thrust_rate'])
print('MTOW: %.0f' % eval_problem['weight:aircraft:MTOW'])

INFO    : Computation finished. Problem outputs written in D:\cdavid\PyCharmProjects\FAST-OAD\notebooks\tutorial\workdir\problem_outputs.xml


NL: NLBGS Converged in 15 iterations
Input max thrust: 70000
Cruise thrust rate: 1.17
MTOW: 73824


The problem has been solved, but you can in log messages and in outputs that the resulting thrust rate is above 1. **Indeed, the specified maximum thrust is too low.**

By the way, though there is a specified constraint in the configuration for having thrust rate between 0. and 1., this constraint does apply only in optimization problems.

You may run the problem again after having increased the maximum thrust, or run an optimization problem that will take it into account.

## 4. Running your first MDO

The configuration file defines a (not very relevant) optimization problem that aims at minimizing the MTOW of aircraft for the defined mission by adjusting the maximum thrust of engines (which has some influence on their fuel consumption, hence the needed block fuel).

*(This run should take a couple of minutes)*

In [93]:
# problem.set_solver_print(level=2)
optim_problem = api.optimize_problem(CONFIGURATION_FILE, overwrite=True)
print('"Optimized" max thrust: %.0f' % optim_problem['propulsion:MTO_thrust'])
print('Cruise thrust rate: %.2f' % optim_problem['propulsion:thrust_rate'])
print('MTOW: %.0f' % optim_problem['weight:aircraft:MTOW'])

NL: NLBGS Converged in 15 iterations


NL: NLBGS Converged in 5 iterations


NL: NLBGS Converged in 21 iterations


NL: NLBGS Converged in 1 iterations


NL: NLBGS Converged in 31 iterations


...

NL: NLBGS Converged in 6 iterations


NL: NLBGS Converged in 4 iterations


NL: NLBGS Converged in 17 iterations


INFO    : Computation finished. Problem outputs written in D:\cdavid\PyCharmProjects\FAST-OAD\notebooks\tutorial\workdir\problem_outputs.xml


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.5501708394459386
            Iterations: 3
            Function evaluations: 13
            Gradient evaluations: 3
Optimization Complete
-----------------------------------
"Optimized" max thrust: 107301
Cruise thrust rate: 0.80
MTOW: 76505
